# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [7]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Get all the source files

In [8]:
# Check the current working directory
print(os.getcwd())

# Collect all the files inside the 'event_data' folder
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/workspace/home
['/workspace/home/event_data/2018-11-10-events.csv', '/workspace/home/event_data/2018-11-20-events.csv', '/workspace/home/event_data/2018-11-09-events.csv', '/workspace/home/event_data/2018-11-11-events.csv', '/workspace/home/event_data/2018-11-07-events.csv', '/workspace/home/event_data/2018-11-27-events.csv', '/workspace/home/event_data/2018-11-21-events.csv', '/workspace/home/event_data/2018-11-15-events.csv', '/workspace/home/event_data/2018-11-17-events.csv', '/workspace/home/event_data/2018-11-06-events.csv', '/workspace/home/event_data/2018-11-24-events.csv', '/workspace/home/event_data/2018-11-23-events.csv', '/workspace/home/event_data/2018-11-04-events.csv', '/workspace/home/event_data/2018-11-30-events.csv', '/workspace/home/event_data/2018-11-29-events.csv', '/workspace/home/event_data/2018-11-01-events.csv', '/workspace/home/event_data/2018-11-16-events.csv', '/workspace/home/event_data/2018-11-02-events.csv', '/workspace/home/event_data/2018-11-13-events.c

#### Pre-processing the files to create the data file csv that will be used for Apache Casssandra tables

In [9]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# Read each line in each file and append them 
# in the emplty list initialised above
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line) 
            
# Collect the data we are interested into a new file called 'event_datafile_new.csv'
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    # write the column headers as the first row of the CSV
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    # write the rest of the data line by line
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))
print('DONE!')

DONE!


In [10]:
# check the number of rows the new file contains
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))


6821


# Part II. Extracting the data prepared above into Cassandra database. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing the data into the Apache Cassandra

In [11]:
# To establish a connection session to a local Cassandra database cluster
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
    session = cluster.connect()
    print('Connected!')
except Exception as e:
    print(e)

Connected!


#### Create Keyspace

In [12]:
# Create a Keyspace called 'project1'
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1 WITH REPLICATION = { 'class': 'SimpleStrategy', 'replication_factor' : 1}
    """)
    print('Keyspace Created!')
except Exception as e:
    print(e)

Keyspace Created!


#### Set Keyspace

In [13]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('project1')
    print('Session SET!')
except Exception as e:
    print(e)

Session SET!


## We will model our data and create three tables that will be optimised for the following three queries

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [14]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# SQL Statement equivalent of Query1. 
## SELECT artist_name, song_title, song_length 
## FROM project1.music_app_history 
## WHERE sessionId = 338 AND itemInSession = 4;

## Create a table for the above query

query1 = """
CREATE TABLE IF NOT EXISTS app_history 
(sessionId INT, itemInSession INT, userId INT, artist TEXT, song TEXT, length FLOAT, 
PRIMARY KEY(sessionId, itemInSession, userId));
"""

try:
    session.execute('DROP TABLE IF EXISTS app_history;')
    session.execute(query1)
    print('app_history CREATED!')
except Exception as e:
    print(e)
                    

app_history CREATED!


In [15]:
# Populate the table with data

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO app_history (sessionId, itemInSession, userId, artist, song, length)"
            query = query + "VALUES (%s, %s, %s, %s, %s, %s)"           
            session.execute(query, (int(line[8]), int(line[3]), int(line[10]), line[0], line[9], float(line[5])))
        print('Inserting COMPLETED!')
except Exception as e:
    print(e)

Inserting COMPLETED!


#### Do a SELECT to verify that the data have been inserted into each table

In [16]:
## Give me the artist, song title and song's length in the music app history that was heard 
## during sessionId = 338, and itemInSession = 4

query_select1 = "select artist, song, length from app_history where sessionId=338 and itemInSession=4;"
try:
    rows = session.execute(query_select1)
    df = pd.DataFrame(rows.current_rows, columns=['Artist', 'Song', 'Length'])
except Exceptioni as e:
    print(e)

## Display the result
df.head()

,Artist,Song,Length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [17]:
# Query2: Give me only the following: name of artist, song (sorted by itemInSession) 
#         and user (first and last name) for userid = 10, sessionid = 182

#SQL equivalent of Query 2: 
# SELECT artist_name, song_title, first_name, last_name FROM project1.music_app_history
# WHERE userid = 10 and sessionid = 182 order by itemInSession, first_name, last_name

## Create a table called user_playlist optimised for Query2

query2 = """
CREATE TABLE IF NOT EXISTS user_playlist 
(userId INT, sessionId INT, artist TEXT, song TEXT, itemInSession INT, firstName TEXT, lastName TEXT, 
PRIMARY KEY(userId, sessionId, itemInSession)) 
WITH CLUSTERING ORDER BY (sessionId ASC, itemInSession ASC);
"""
try:
    session.execute('DROP TABLE IF EXISTS user_playlist;')
    session.execute(query2)
    print('user_playlist CREATED!')
except Exception as e:
    print(e)
                    

user_playlist CREATED!


In [18]:
## Populate the table with data 

file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            query = "INSERT INTO user_playlist (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
            
            session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
        print('Inserting user_playlist COMPLETED!')
except Exception as e:
    print(e)

Inserting user_playlist COMPLETED!


In [19]:
# Test the table by querying the table.
## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) 
## for userid = 10, sessionid = 182

query_select2 = "select artist, song, firstName, lastName FROM user_playlist WHERE userId = 10 AND sessionId = 182;"
try:
    rows = session.execute(query_select2)
    df = pd.DataFrame(rows.current_rows, columns=['Artist', 'Song', 'First Name', 'Last Name'])
except Exceptioni as e:
    print(e)

## Display the result    
df.head()

,Artist,Song,First Name,Last Name
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


In [20]:
# Query3: Give me every user name (first and last) in my music app history who listened to the song 
# 'All Hands Against His Own'

# SQL equivalent of the above query. 
## select first_name, last_name from music_app_history 
## where song_title = 'All Hands Against His Own'

## Create a table called music_history for the above query
query3 = """
CREATE TABLE IF NOT EXISTS music_history 
(userId INT, sessionId INT, artist TEXT, song TEXT, itemInSession INT, firstName TEXT, lastName TEXT, 
PRIMARY KEY(song, userId)) WITH CLUSTERING ORDER BY (userId ASC);
"""
try:
    session.execute('DROP TABLE IF EXISTS music_history;')
    session.execute(query3)
    print('music_history CREATED!')
except Exception as e:
    print(e)

music_history CREATED!


In [21]:
file = 'event_datafile_new.csv'

try:
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader) # skip header
        for line in csvreader:
            ## Prepare and insert data into the Cassandra table
            query = "INSERT INTO music_history (userId, sessionId, artist, song, itemInSession, firstName, lastName)"
            query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
            
            session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], int(line[3]), line[1], line[4]))
        print('Inserting music_history COMPLETED!')
except Exception as e:
    print(e)
                    

Inserting music_history COMPLETED!


In [22]:
## Give me every user name (first and last) in my music app history who listened to the song 
## 'All Hands Against His Own'
query_select3 = "select firstName, lastName from music_history where song = 'All Hands Against His Own';"
try:
    rows = session.execute(query_select3)
    df = pd.DataFrame(rows.current_rows, columns=['First Name', 'Last Name'])
except Exceptioni as e:
    print(e)
    
## Display the result
df.head()

,First Name,Last Name
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [23]:
## Drop the table before closing out the sessions
try:
    rows = session.execute("drop table if exists app_history;")
    rows = session.execute("drop table if exists user_playlist;")
    rows = session.execute("drop table if exists music_history;")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()